In [107]:
import gmaps
from api_keys import gkey
import gmaps.geojson_geometries
import requests
import json
from matplotlib.cm import Wistia
from matplotlib.colors import to_hex
import pandas as pd
import matplotlib.pyplot as plt

gmaps.configure(api_key=gkey)

In [84]:
zip_geojson = requests.get('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/tx_texas_zip_codes_geo.min.json').content
zip_geojson = json.loads(zip_geojson)

In [85]:
price_df = pd.read_csv('../data/rawdata/Property_data.csv')
price_df.head()

,zip,address,lat,lng,house area,lot area,high,low,estimate,month value change,property tax
0,77004,"3327 FRANCIS ST, HOUSTON, TX 77004",29.728969,-95.354886,792.0,5001.0,100859,84504,90864,87.0,72000.0
1,77004,"1820 OAKDALE ST, HOUSTON, TX 77004",29.723462,-95.381033,2892.0,5500.0,635167,531231,577425,1263.0,383698.0
2,77004,"2220 MACGREGOR WAY, HOUSTON, TX 77004",29.715995,-95.380015,2813.0,8638.0,526769,425281,483274,7871.0,275131.0
3,77004,"2402 WENTWORTH ST, HOUSTON, TX 77004",29.724232,-95.373084,2748.0,6599.0,445146,316883,377242,-1526.0,182314.0
4,77004,"2424 ISABELLA ST, HOUSTON, TX 77004",29.729207,-95.368853,1470.0,5776.0,267123,241683,254403,1862.0,183358.0


In [86]:
zip_coords = pd.read_csv("../data/rawdata/zipcode_lat_lngs.csv")

In [87]:
grouper = price_df.groupby("zip")

In [88]:
grouper = price_df.groupby("zip")

price = pd.DataFrame(grouper[["estimate","house area","lot area","high","low","month value change","property tax"]].mean())
price = price.reset_index()
print(len(price.index))
price.head()

131


,zip,estimate,house area,lot area,high,low,month value change,property tax
0,77003,304618.520,1728.452716,3171.167339,329690.960,281187.204,745.221328,2.456000e+05
1,77004,332700.348,2040.959839,6091.640562,365302.716,302881.670,-647.458918,2.551451e+05
2,77005,1185270.596,3363.540000,7277.734406,1282708.420,1099375.018,10074.394790,1.194511e+06
3,77006,642616.528,2387.026104,4935.989518,702146.308,591382.956,3892.106212,5.803662e+05
4,77007,528380.128,2360.635271,3457.987854,568855.332,490881.380,-1220.452906,5.022877e+05


In [89]:
have_zips = zip_coords[zip_coords["ZIP"].isin(list(price["zip"]))]
have_zips = have_zips.reset_index(drop=True)
print(len(have_zips))
have_zips.head()

131


,ZIP,LAT,LNG
0,77003,29.749778,-95.345885
1,77004,29.724893,-95.363752
2,77005,29.718435,-95.423555
3,77006,29.740970,-95.391301
4,77007,29.771545,-95.411083


In [90]:
tot = pd.merge(price, have_zips,left_on="zip",right_on="ZIP")
tot = tot.drop("ZIP",axis=1)
tot.head()

,zip,estimate,house area,lot area,high,low,month value change,property tax,LAT,LNG
0,77003,304618.520,1728.452716,3171.167339,329690.960,281187.204,745.221328,2.456000e+05,29.749778,-95.345885
1,77004,332700.348,2040.959839,6091.640562,365302.716,302881.670,-647.458918,2.551451e+05,29.724893,-95.363752
2,77005,1185270.596,3363.540000,7277.734406,1282708.420,1099375.018,10074.394790,1.194511e+06,29.718435,-95.423555
3,77006,642616.528,2387.026104,4935.989518,702146.308,591382.956,3892.106212,5.803662e+05,29.740970,-95.391301
4,77007,528380.128,2360.635271,3457.987854,568855.332,490881.380,-1220.452906,5.022877e+05,29.771545,-95.411083


In [91]:
tot.to_csv("../data/rawdata/average_data.csv",index = False)

In [92]:
needed_zips = []
needed_zips = list(have_zips["ZIP"])

for i in needed_zips:
    i = str(i)

In [93]:
list_index = []

for i in range(len(zip_geojson['features'])):
    if zip_geojson['features'][i]['properties']['ZCTA5CE10'] not in str(needed_zips):
        list_index.append(i)
        
for index in sorted(list_index, reverse=True): 
    del zip_geojson['features'][index]
len(zip_geojson['features'])

131

In [94]:
sort_list = []
for i in range(len(zip_geojson["features"])):
    comp = int(zip_geojson['features'][i]['properties']['ZCTA5CE10'])
    sort_list.extend(tot.loc[tot["zip"] == comp].index)
tot = tot.iloc[sort_list]
tot = tot.reset_index(drop=True)
tot.head()

,zip,estimate,house area,lot area,high,low,month value change,property tax,LAT,LNG
0,77099,153013.132,1710.190763,7220.537815,162568.672,142455.332,611.520000,127688.814889,29.670869,-95.585990
1,77389,302047.312,2587.334004,14305.885481,323906.618,280033.594,687.627255,260105.826000,30.123469,-95.517064
2,77586,306169.674,2505.973948,11979.337398,330028.422,283307.076,-761.746000,267028.967936,29.583816,-95.036166
3,77020,134260.098,1201.845528,5459.108434,152238.644,121889.014,-2146.404810,77215.792339,29.773179,-95.314327
4,77013,140875.606,1512.776000,11796.918000,150485.660,130264.036,989.882000,92886.230000,29.795268,-95.233977


In [95]:
#This section is unused, ignore it.
# We will need to scale the GINI values to lie between 0 and 1
#min_gini = min(harris_zips)
#max_gini = max(harris_zips)
#gini_range = max_gini - min_gini

# def calculate_color(gini):
#     """
#     Convert the GINI coefficient to a color
#     """
#     # make gini a number between 0 and 1
#     normalized_gini = (gini - min_gini) / gini_range

#     # invert gini so that high inequality gives dark color
#     inverse_gini = 1.0 - normalized_gini

#     # transform the gini coefficient to a matplotlib color
#     mpl_color = viridis(inverse_gini)

#     # transform from a matplotlib color to a valid CSS color
#     gmaps_color = to_hex(mpl_color, keep_alpha=False)

#     return gmaps_color

def calculate_colors(input_list):
    _min = tot["estimate"].min()
    _max = tot["estimate"].max()
    _range = _max - _min
    
    result = []
    
    for i in input_list:
        norm = (i - _min) / _range
        
        inverse = 1.0 - norm
        mpl_color = Wistia(inverse)
        gmaps_color = to_hex(mpl_color, keep_alpha=False)
        
        result.append(gmaps_color)
        
    return result

In [96]:
colors = calculate_colors(tot['estimate'])

In [109]:
fig = gmaps.figure(map_type = 'HYBRID')

gini_layer = gmaps.geojson_layer(
    zip_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
# zip_layer = gmaps.geojsown_layer(zip_geojson)
#restaraunt_layer = gmaps.heatmap_layer(have_zips[['LAT', 'LNG']], weights = price["estimate"])
# fig.add_layer(restaraunt_layer)
# fig.add_layer(zip_layer)
fig.add_layer(gini_layer)
fig

Figure(layout=FigureLayout(height='420px'))